In [0]:
# Load the table
data = spark.sql("select * from default.reviews_train")

data = data.sample(False, 0.01, seed=0)

data = data.cache()

print((data.count(), len(data.columns)))

(31298, 11)


In [0]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, Stemmer

from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, StringIndexer, SQLTransformer, IndexToString, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier


# convert text column to spark nlp document
document_assembler = DocumentAssembler() \
    .setInputCol("reviewText") \
    .setOutputCol("document")


# convert document to array of tokens
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
 
# clean tokens 
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

# stems tokens to bring it to root form
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

# Convert custom document structure to array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

# Generate Term Frequency
tf = CountVectorizer(inputCol="token_features", outputCol="rawFeatures", vocabSize=10000, minTF=1, minDF=50, maxDF=0.40)

# Generate Inverse Document Frequency weighting
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures", minDocFreq=5)

# Combine all features into one final "features" column
assembler = VectorAssembler(inputCols=["verified", "overall", "idfFeatures"], outputCol="features")

# Machine Learning Algorithm
#ml_alg  = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.0)
ml_alg  = RandomForestClassifier(numTrees=100, featureSubsetStrategy="auto", impurity='gini', maxDepth=4, maxBins=32)

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            tf,
            idf,
            assembler,
            ml_alg])
            

In [0]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed = 47)

In [0]:
pipeline_model = nlp_pipeline.fit(trainingData)

In [0]:
predictions =  pipeline_model.transform(testData)
display(predictions)

reviewID overall verified reviewTime reviewerID asin reviewerName reviewText summary unixReviewTime label document token normalized cleanTokens stem token_features rawFeatures idfFeatures features rawPrediction probability prediction 252 5.0 false 01 23, 2017 A1YTZAZZ1L32RL 0002005263 Chuck Ok reading Ok 1485129600 0 List(List(document, 0, 9, Ok reading, Map(sentence -> 0), List())) List(List(token, 0, 1, Ok, Map(sentence -> 0), List()), List(token, 3, 9, reading, Map(sentence -> 0), List())) List(List(token, 0, 1, Ok, Map(sentence -> 0), List()), List(token, 3, 9, reading, Map(sentence -> 0), List())) List(List(token, 0, 1, Ok, Map(sentence -> 0), List()), List(token, 3, 9, reading, Map(sentence -> 0), List())) List(List(token, 0, 1, ok, Map(sentence -> 0), List()), List(token, 3, 9, read, Map(sentence -> 0), List())) List(ok, read) Map(vectorType -> sparse, length -> 2215, indices -> List(6, 398), values -> List(1.0, 1.0)) Map(vectorType -> sparse, length -> 2215, indices -> List(6, 398), values -> List(1.83014163987146, 4.086274259697965)) Map(vectorType -> sparse, length -> 2217, indices -> List(1, 8, 400), values -> List(5.0, 1.83014163987146, 4.086274259697965)) Map(vectorType -> dense, length -> 2, values -> List(84.86229554966609, 15.137704450333944)) Map(vectorType -> dense, length -> 2, values -> List(0.8486229554966607, 0.1513770445033394)) 0.0 502 3.0 false 07 2, 2014 A18LVHY887HL23 1933682612 AJ These are good but as others have said, they're better if you have a regular set of refrigerator magnets because this set, while containing a great mix of profanity, is lack a lot of structural content so it's hard to really make a sentence which takes away some of the novelty of having these magnets in the first place. They should sell this as a bundle with a regular set because there are only so many f**** you can put in a row before it just looks stupid. As far as profanity seekers, this set is for you. Lots of profane words, I would list some if I could. I even learned a few new words from the set so it's both fun and educational :-P Good but... 1404259200 1 List(List(document, 0, 644, These are good but as others have said, they're better if you have a regular set of refrigerator magnets because this set, while containing a great mix of profanity, is lack a lot of structural content so it's hard to really make a sentence which takes away some of the novelty of having these magnets in the first place. They should sell this as a bundle with a regular set because there are only so many f**** you can put in a row before it just looks stupid. As far as profanity seekers, this set is for you. Lots of profane words, I would list some if I could. I even learned a few new words from the set so it's both fun and educational :-P, Map(sentence -> 0), List())) List(List(token, 0, 4, These, Map(sentence -> 0), List()), List(token, 6, 8, are, Map(sentence -> 0), List()), List(token, 10, 13, good, Map(sentence -> 0), List()), List(token, 15, 17, but, Map(sentence -> 0), List()), List(token, 19, 20, as, Map(sentence -> 0), List()), List(token, 22, 27, others, Map(sentence -> 0), List()), List(token, 29, 32, have, Map(sentence -> 0), List()), List(token, 34, 37, said, Map(sentence -> 0), List()), List(token, 38, 38, ,, Map(sentence -> 0), List()), List(token, 40, 46, they're, Map(sentence -> 0), List()), List(token, 48, 53, better, Map(sentence -> 0), List()), List(token, 55, 56, if, Map(sentence -> 0), List()), List(token, 58, 60, you, Map(sentence -> 0), List()), List(token, 62, 65, have, Map(sentence -> 0), List()), List(token, 67, 67, a, Map(sentence -> 0), List()), List(token, 69, 75, regular, Map(sentence -> 0), List()), List(token, 77, 79, set, Map(sentence -> 0), List()), List(token, 81, 82, of, Map(sentence -> 0), List()), List(token, 84, 95, refrigerator, Map(sentence -> 0), List()), List(token, 97, 103, magnets, Map(sentence -> 0), List()), List(token, 105, 111, because, Map(sentence -> 0), List()), List(token, 113, 116, thi

In [0]:
predictions.groupBy("label").count().show()
predictions.groupBy("prediction").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 1130|
|    0| 4993|
+-----+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 6096|
|       1.0|   27|
+----------+-----+



In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
pre_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
rec_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
pr_evaluator  = BinaryClassificationEvaluator(metricName="areaUnderPR")
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

print("Test Accuracy       = %g" % (acc_evaluator.evaluate(predictions)))
print("Test Precision      = %g" % (pre_evaluator.evaluate(predictions)))
print("Test Recall         = %g" % (rec_evaluator.evaluate(predictions)))
print("Test areaUnderPR    = %g" % (pr_evaluator.evaluate(predictions)))
print("Test areaUnderROC   = %g" % (auc_evaluator.evaluate(predictions)))

Test Accuracy       = 0.81692
Test Precision      = 0.789733
Test Recall         = 0.81692
Test areaUnderPR    = 0.4987
Test areaUnderROC   = 0.816456
